In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import six
project_path = os.getcwd() + '/../'
sys.path.append(project_path + "utils")

import sklearn.metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.grid_search import GridSearchCV
import sklearn
from sklearn.externals import joblib
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

/Users/uchihashi_k/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns

import function
import preprocessing
import local_bayes
import visualize
import metrics

In [3]:
sys.path.append(project_path + "models/cnn")
import cnn

# nearest points

In [4]:
save_data_dir = '../data/bookmark/'
train_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_train.npz')
test_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_test.npz')

In [5]:
new_tr = train_0_0['x'][:1000]
new_te = test_0_0['x'][0]

In [6]:
localbayes = local_bayes.LocalBayes(nn_num=20, data_dim=30)

In [7]:
nn_dist_ls, nn_ts_ls = localbayes.get_nearest_n(train_0_0['x'], new_te)

758

In [8]:
len(nn_ts_ls)

20

In [9]:
numpy.array(nn_ts_ls)

array([[ 2,  1,  1,  0,  7,  3,  4,  7, 11,  6,  4,  4,  7,  5,  5,  7,  1,
         3,  5,  5,  7,  0,  1,  3,  2,  2,  4,  3,  1,  4],
       [ 2,  1,  2,  8,  1,  2,  2,  8,  5,  1,  4,  4,  8, 10,  2,  2,  0,
         2,  2,  2,  2,  1,  2,  2,  1,  0,  2,  2,  6,  0],
       [ 3,  2,  3,  5,  7,  1,  7,  1,  1,  1,  1,  2,  4,  1,  1,  1,  0,
         1,  2,  4,  3,  1,  3,  1,  2,  3,  0,  4,  0,  1],
       [ 0,  2,  1,  6, 16, 13,  3,  5,  6,  1,  2,  1,  5,  0,  1,  3,  7,
         1,  1,  1,  1,  1,  1,  1,  2,  1,  0,  3,  0,  1],
       [ 1,  9,  6,  2, 14, 10,  5,  5,  1,  2,  4,  1,  2,  1,  6,  0,  4,
         5,  3,  2,  0,  0,  3,  4,  4,  3,  2,  0,  2,  1],
       [ 0,  2,  5,  2,  6,  4,  5,  1,  4,  3,  2,  1,  4,  1,  0,  1,  3,
         5,  2,  1,  3,  2,  2,  1,  2,  1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  2,  5, 11,  4,  1,  0,  2,  0,  2,  2,  2,  3,  1,
         1,  1,  1,  1,  6,  0,  3,  2,  3,  1,  3,  4,  0],
       [ 0,  0,  0,  0,  7, 13, 22,  8,  

## prediction

### gradient boosting decision tree

In [10]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
max_depth = 4
n_estimators = 5000

In [12]:
bdt = joblib.load('../models/gbdt/results/bdt_%s_%s.pkl' % (max_depth, n_estimators))

In [13]:
gbdt_pred = bdt.predict_proba(numpy.array(nn_ts_ls))
gbdt_pred = gbdt_pred[:, 1].tolist()
gbdt_pred

[0.49595700562528316,
 0.4963279326980058,
 0.5018216740450403,
 0.5001240860643253,
 0.5020297148593013,
 0.5006630041875523,
 0.49494918737213006,
 0.48509078479105927,
 0.49769480945527955,
 0.49882025072187747,
 0.49312866101663677,
 0.4945560017422757,
 0.5019556691731124,
 0.5015629959725285,
 0.5009917399909998,
 0.49657133035668094,
 0.49769480945527955,
 0.49734666478262257,
 0.4833935677247825,
 0.4968898831320494]

### CNN

In [14]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import cuda
from chainer import optimizers
from chainer import serializers
from chainer import Link, Chain, ChainList

import pandas as pd
from ggplot import *

In [15]:
save_model_dir = '../models/cnn/results/awesome_100/'

In [16]:
sys.path.append(project_path + "models/cnn")
import cnn_net

In [17]:
N_test = numpy.array(nn_ts_ls).shape[0]
data_dim = test_0_0['x'].shape[1]

In [18]:
model = L.Classifier(cnn_net.tsCNN_net(data_dim))

In [19]:
serializers.load_npz(save_model_dir + "mlp.model", model)

In [20]:
def forward(x_data, model):
    x = chainer.Variable(x_data)
    h = F.max_pooling_2d(F.relu(model.conv1(x)), 2)
    h = F.max_pooling_2d(F.relu(model.conv2(h)), 2)
    h = F.relu(model.l1(h))
    y = model.l2(h)

    return y

In [21]:
nn_x = numpy.array(nn_ts_ls).reshape((N_test, 1, 1, data_dim))

In [22]:
cnn_pred = []
for i in range(0, N_test):
    x = np.asarray(nn_x[i]).reshape((1, 1, 1, data_dim)).astype(numpy.float32)
    
    pred_ = F.softmax(model.predictor(chainer.Variable(x)))
    cnn_pred.append(pred_.data[0][1])

In [23]:
cnn_pred

[0.0026402085,
 9.3086167e-10,
 0.99981683,
 0.99943095,
 0.99990177,
 0.98906881,
 5.6098847e-06,
 1.2144666e-16,
 0.00092398445,
 0.0015024451,
 0.0041209068,
 1.322026e-07,
 0.99985313,
 0.99979764,
 0.99630016,
 0.0026082713,
 0.00092398445,
 0.024740428,
 2.0070509e-08,
 0.0031541879]

## local bayes estimation

In [99]:
localbayes = local_bayes.LocalBayes(nn_num=20, data_dim=30, max_dist=50)

In [29]:
learner_name_ls = ['cnn', 'gbdt']
pred_proba_dict = {}
pred_proba_dict.update({'cnn':cnn_pred})
pred_proba_dict.update({'gbdt':gbdt_pred})

In [32]:
model_dict = {}
model_dict.update({'cnn':model})
model_dict.update({'gbdt':bdt})

In [114]:
len(test_0_0['x'])

2378

In [115]:
train_data_num = 500
test_data_num = len(test_0_0['x'])
save_result_dir = '../results/nn_num=%s_train_data_num=%s_test_data_num=%s/' % (nn_num, train_data_num, test_data_num)
# os.mkdir(save_result_dir)

In [116]:
lb_pred_ls, w_dict_ls = [], []
for i in xrange(test_data_num):
    test_data = test_0_0['x'][i]
    nn_dist_ls, nn_ts_array = localbayes.get_nearest_n(train_0_0['x'][:train_data_num], test_data)
    nn_pred_dict = localbayes.cal_prediction_nearest_n(test_data, nn_ts_array, learner_name_ls, model_dict)
    
    w_dict = localbayes.local_bayes_estimation(nn_pred_dict, learner_name_ls)
    w_dict_ls.append(w_dict)
    
    lb_pred = localbayes.local_bayes_predict(test_data, learner_name_ls, w_dict, model_dict)
    lb_pred_ls.append(lb_pred)
    
    sys.stdout.write('\r%d' % i)
    sys.stdout.flush()
numpy.savez(save_result_dir + 'result.npz', w=w_dict_ls, pred=lb_pred_ls)

2377

IOError: [Errno 2] No such file or directory: '../results/nn_num=20_train_data_num=500_test_data_num=2378/result.npz'

In [110]:
result = numpy.load(save_result_dir + 'result.npz')

In [113]:
result['pred']

array([  4.92004619e-01,   5.00054838e-01,   1.35121895e-02,
         1.38529181e-03,   5.03695067e-01,   4.62573896e-01,
         9.94276583e-01,   1.00000000e+00,   7.15879786e-22,
         5.01461328e-01])

In [102]:
lb_pred = localbayes.local_bayes_predict(test_0_0['x'][6], learner_name_ls, w_dict_ls[6], model_dict)
lb_pred

0.99427658319473267

In [103]:
w_dict_ls

[{'cnn': 0.0, 'gbdt': 1.0},
 {'cnn': 0.0, 'gbdt': 1.0},
 {'cnn': 1.0, 'gbdt': 0.0},
 {'cnn': 1.0, 'gbdt': 0.0},
 {'cnn': 0.0, 'gbdt': 1.0},
 {'cnn': 0.0, 'gbdt': 1.0},
 {'cnn': 1.0, 'gbdt': 0.0},
 {'cnn': 1.0, 'gbdt': 0.0},
 {'cnn': 1.0, 'gbdt': 0.0},
 {'cnn': 0.0, 'gbdt': 1.0}]

In [104]:
lb_pred_ls

[0.49200461940550261,
 0.50005483764867875,
 0.013512189500033855,
 0.0013852918054908514,
 0.50369506668047115,
 0.46257389601869942,
 0.99427658319473267,
 1.0,
 7.158797856732882e-22,
 0.50146132757516548]

In [105]:
test_0_0['y'][:10]

array([0, 0, 1, 0, 1, 0, 1, 1, 0, 1])